# Gauss-Jacobi and Gauss-Seidel
*July 25th, 2024*

### Algorithms

General iteration algorithm: $A = M + N$, we have the update formula $Mx^{(k + 1)} + Nx^{(k)} = b$.

Gauss-Jacobi uses $M = D$, Gauss-Seidel uses $M = L + D$.

Can rewrite update formula as $x^{(k + 1)} = M^{-1}(b - Nx^{(k)}) = x^{(k)} + M^{-1}(b - Ax^{k})$.

### Convergence

#### Gershgorin circle theorem

Let $A$ be a complex-valued matrix, then the claim here is that every eigenvalue of $A$ lies in some disc of the form $\overline{B(a_{ii}, R_i)}$, where $R_i = \sum_{j \ne i} |a_{ij}|$.

The proof is simple: Let $\lambda, x$ be an eigenpair of $A$. Then, we have $Ax = \lambda x$, in which case we have $(Ax)_i = a_{ij}x_j = \lambda x_i$. Then, $\sum_{j \ne i} a_{ij}x_j  = (\lambda - a_{ii})x_i$. Now, choose $i$ such that $x_i$ has maximal magnitude, and we obtain $$|\lambda - a_{ii}| = \left|{\sum_{i \ne j} a_{ij}\frac{x_j}{x_i}}\right| \le \sum_{j \ne i} |{a_{ij}}|\left|{\frac{x_j}{x_i}}\right| \le \sum_{j \ne i} |x_j|$$.

#### GJ Convergence

Asssuming $A$ is diagonally dominant, then the error propagator matrix $I - D^{-1}A$ is of the form $$I - D^{-1}A = \begin{bmatrix} 0 & \frac{a_{ij}}{a_{11}} & \cdots & \frac{a_{1n}}{a_{11}} \\ a_{} & \ddots \\ & & \ddots \\  \end{bmatrix}$$. The non-diagonal entries of this matrix for every row sum up to less than $1$, so applying the circle theorem, we have that the all the eigenvalues of this matrix are less than $1$, and thus the matrix norm is strictly less than $1$.

Now, given the error update formula $e_{k + 1} = (I - D^{-1}A)e_k$, by seeing that $I - D^{-1}A$ has norm $\alpha < 1$, we see that $\| e_k \| < \alpha^k \|e_0\| \to 0$.


In [1]:
import pandas
import numpy as np
import plotly.express as px
import numpy.linalg as LA

In [2]:
# Relaxation Methods
def gauss_jacobi(A: np.array, b: np.array, x0: np.array, term_cond, weight = 1):
    n = np.shape(A)[0]
    D = np.diag(np.diag(A))
    M = D
    curr = x0
    prev = None
    # print(curr, prev)
    while not term_cond(A, b, prev, curr):
        prev = curr
        curr = prev + weight * np.matmul(LA.inv(M), b - np.matmul(A, prev))
    return curr

def gauss_seidel(A: np.array, b: np.array, x0: np.array, term_cond, weight = 1):
    L = np.tril(A, -1)
    D = np.diag(np.diag(A))
    M = L + D
    curr = x0
    prev = None
    # print(curr, prev)
    while not term_cond(A, b, prev, curr):
        prev = curr
        curr = prev + weight * np.matmul(LA.inv(M), b - np.matmul(A, prev))
    return curr


def small_diff(A, b, prev, curr):
    if not isinstance(prev, np.ndarray): return False
    return LA.norm(prev - curr) < 1E-1

def small_res(A, b, prev, curr):
    return LA.norm(np.matmul(A, curr) - b) < 1E-3


GJ example with discretization of Jacobian: $$A = \begin{bmatrix} 2 & -1 & & & \\ -1 & 2 & -1 & \\ & \ddots & \ddots & \ddots \\ & &-1 & 2 & -1 \\ & & & -1 & 2\end{bmatrix}$$

In [3]:
N = 10
A = np.diag(np.full(N, 2.)) + np.diag(np.full(N - 1, -1), -1) + np.diag(np.full(N - 1, -1), 1) 
A

array([[ 2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  2., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  2., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  2., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  2.]])

In [4]:
b = np.random.rand(N)

print(f'b = {b}\n')

gj = gauss_jacobi(A, b, np.zeros(N), small_diff)
exact = np.matmul(np.linalg.inv(A), b)
print(f'Gauss-Jacobi:\n{gj}\n\nExact:\n{exact}\n\nError:\n{exact - gj}\n\nError Norm:\n{LA.norm(exact - gj)}')

b = [0.18791153 0.08938634 0.74160803 0.03782202 0.56117419 0.61926261
 0.41555456 0.27762023 0.82720707 0.82310648]

Gauss-Jacobi:
[1.57347361 2.98710306 4.3420067  5.03059627 5.73300544 5.97286122
 5.649727   5.00167603 4.11902522 2.46304106]

Exact:
[1.84701532 3.50611911 5.07583655 5.90394597 6.69423337 6.92334657
 6.53319716 5.7274932  4.64416901 2.73363774]

Error:
[0.27354171 0.51901605 0.73382985 0.8733497  0.96122792 0.95048535
 0.88347017 0.72581717 0.52514379 0.27059668]

Error Norm:
2.264764634008729


Why does this converge even when $A$ is not diagonally dominant? Because diagonal dominance is not a necessary condition for the matrix norm of $I - D^{-1}A$ to be less than $1$. 

In [5]:
LA.eigvals(np.matmul(LA.inv(np.diag(np.diag(A))), A)) # eigenvalues of D^{-1}A

array([1.95949297, 1.84125353, 1.65486073, 1.41541501, 1.14231484,
       0.85768516, 0.04050703, 0.15874647, 0.34513927, 0.58458499])

Now for a GS example with a matrix with 10s on the diagonal and -1 on the upper and lower diagonal.

In [6]:
N = 20
A = np.diag(np.full(N, 10.)) + np.diag(np.full(N - 1, -1), -1) + np.diag(np.full(N - 1, -1), 1) 
b = np.random.rand(N)

gs = gauss_seidel(A, b, np.zeros(N), small_diff)
exact = np.matmul(LA.inv(A), b)
print(f'Gauss-Seidel:\n{gs}\n\nExact:\n{exact}\n\nError:\n{exact - gs}\n\nError Norm:\n{LA.norm(exact - gs)}')

Gauss-Seidel:
[0.10289468 0.04752151 0.05483818 0.0993159  0.09992244 0.08163282
 0.06808903 0.07799194 0.06504715 0.04681129 0.07983202 0.05576333
 0.10830505 0.08198578 0.02434497 0.09365275 0.03229393 0.0254352
 0.01315615 0.04039648]

Exact:
[0.10350398 0.04864713 0.05606533 0.10037594 0.10081148 0.08258629
 0.0689306  0.07862631 0.06597621 0.04755907 0.08105984 0.05683146
 0.10875509 0.08300517 0.02486926 0.09400617 0.03249179 0.02587529
 0.01324219 0.04040509]

Error:
[6.09300415e-04 1.12561944e-03 1.22714565e-03 1.06003587e-03
 8.89041709e-04 9.53471091e-04 8.41565898e-04 6.34369167e-04
 9.29056530e-04 7.47782257e-04 1.22781677e-03 1.06813722e-03
 4.50041911e-04 1.01938866e-03 5.24295214e-04 3.53418007e-04
 1.97862503e-04 4.40089208e-04 8.60392166e-05 8.60392166e-06]

Error Norm:
0.003609785224978833


### Discretization of ODE

In [7]:
def gauss_jacobi_visual(A: np.array, b: np.array, x0: np.array, term_cond, x, weight = 1):
    n = np.shape(A)[0]
    D = np.diag(np.diag(A))
    M = D
    curr = x0
    prev = None
    # print(curr, prev)
    i = 0
    uis = []
    while not term_cond(A, b, prev, curr) and i < 501:
        prev = curr
        curr = prev + weight * np.matmul(LA.inv(M), b - np.matmul(A, prev))
        
        if (i % 5 == 0):
            uis.append(curr)
        i += 1
    return uis

def second_order_ode_visualization(a = 0, b = 1, c = 0, f = lambda x: 0, N: int = 10, term_cond = small_diff):
    x = np.fromfunction(lambda i: (b - a)/N * i, shape = [N + 1])
    h = (b - a)/N
    u0 = np.random.rand((N + 1))

    N = int(N)

    b1 = np.zeros(N + 1)
    b1[0] = 0
    b1[N] = 0
    # [u''(xi) = u(x[i + 1]) - 2u(x[i]) + u[x[i - 1]]]
    # [cu = cu([xi])]
    A1 = np.diag(np.full((N + 1), -2/(h * h) + c))
    A2 = np.diag(np.full((N), 1/(h * h)), -1)
    A3 = np.diag(np.full((N), 1/(h * h)), 1)
    A = A1 + A2 + A3
    A[0,0] = 1
    A[0,1] = 0
    A[-1,-1] = 1
    A[-1,-2] = 0
    # print(A)

    return x, gauss_jacobi_visual(A, b1, u0, term_cond, x, weight = 0.75)

x, uis = second_order_ode_visualization(N = 100, c = 0, b=  1, term_cond=lambda a, b, c, d: False)
# px.line(x = x, y = uis)

df = pandas.DataFrame(data = uis).reset_index()
df = pandas.melt(df, id_vars=['index'])
df['variable'] = df['variable'].map(lambda i: x[i])
df = df.rename(columns = {'index': 'iteration'})
df['iteration'] = df['iteration'].map(lambda i: i * 5)

df

,iteration,variable,value
0,0,0.0,8.521814e-02
1,5,0.0,8.322084e-05
2,10,0.0,8.127035e-08
3,15,0.0,7.936557e-11
4,20,0.0,7.750544e-14
...,...,...,...
10196,480,1.0,1.578193e-291
10197,485,1.0,1.541204e-294
10198,490,1.0,1.505082e-297
10199,495,1.0,1.469807e-300


In [8]:
fig = px.line(df, x = 'variable', y = 'value', animation_frame='iteration')
ys = [np.exp(xi) for xi in x]
fig.add_scatter(x = x, y = ys, name= "exact: e^x")
fig